In [1]:
import sys
import json
import requests
import random

import torch as th
from torch import nn
import syft as sy

sy.create_sandbox(globals(), verbose=False)


/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Setting up Sandbox...
Done!


In [2]:
from syft.codes import MSGTYPE
from syft import serde

class CommandCreator():
    
    def __init__(self, id):
        self.id = id
        
    def send_msg(self, msg_type, message, location):
        # Step 0: combine type and message
        message = (msg_type, message)

        # Step 1: serialize the message to simple python objects
        bin_message = serde.serialize(message)
        
        return bin_message
    
    def send_command(self, recipient, message):
        """
        Send a command through a message to a recipient worker
        :param recipient:
        :param message:
        :return:
        """

        response = self.send_msg(MSGTYPE.CMD, message, location=recipient)

        return response

In [3]:
import msgpack

In [6]:
x = th.tensor([1,2,3,4]).create_pointer()
x.owner = CommandCreator("steve")
y = x + x
msgpack.loads(y[1:])

[2,
 [1,
  [2,
   [b'__add__',
    [10, [36593675293, 89052606580, b'me', None, [4]]],
    [2, [[10, [36593675293, 89052606580, b'me', None, [4]]]]]]]]]

In [4]:
sy.serde.deserialize(y)

KeyError: 'Tensor "27412296840" not found on worker "me"!!! You just tried to interact with an object ID:27412296840 on worker me which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [2]:
sy.local_worker.id = "bob"# + str(random.randint(0,1e10))

In [3]:
url = "http://127.0.0.1:5000"

In [4]:
# request model
res = requests.get(url+"/get_model")
m = sy.serde.deserialize(res.content)
m

tensor([[0.],
        [0.]])

In [5]:
data = th.ones(4,2)
ptr = data.create_pointer()
ptr = sy.serde.serialize(ptr)

In [6]:
# request model
res = requests.post(url+"/send_data",data=ptr,headers={'Content-Type': 'application/octet-stream'})
res

<Response [200]>

In [7]:
m = sy.serde.deserialize(res.content)
m

tensor([[0.],
        [0.]])

In [29]:
data.mm(m)

tensor([[0.],
        [0.],
        [0.],
        [0.]])

In [16]:
# Iniitalize A Toy Model
model = nn.Linear(2, 1, bias=False)
model.send(bob)

Linear(in_features=2, out_features=1, bias=False)

In [11]:
bob.log_msgs = True

In [17]:
bob.msg_history

[b'0\x92\x02\x92\x02\x92\x01\x94\xcf\x00\x00\x00\t\x8c\x8d\xd8\x06\x96\xcf\x00\x00\x00\x05\xadw\xe6*\xda\x01_\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02ctorch._utils\n_rebuild_tensor_v2\nq\x00((X\x07\x00\x00\x00storageq\x01ctorch\nFloatStorage\nq\x02X\x0f\x00\x00\x00140431083661328q\x03X\x03\x00\x00\x00cpuq\x04K\x02Ntq\x05QK\x00K\x01K\x02\x86q\x06K\x02K\x01\x86q\x07\x89ccollections\nOrderedDict\nq\x08)Rq\ttq\nRq\x0b.\x80\x02]q\x00X\x0f\x00\x00\x00140431083661328q\x01a.\x02\x00\x00\x00\x00\x00\x00\x00\x83\xed\x1d?\xee&\xd5>\xc0\xc0\xc0\xc0\xc3\xc0',
 b"0\x92\x02\x92\x02\x92\x01\x94\xcf\x00\x00\x00\x10\xc5o\x0b\x1b\x96\xcf\x00\x00\x00\x0b\x91r.\x86\xda\x01_\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00